### Load Main Libraries

In [3]:
from datetime import datetime, timedelta
from mplsoccer import VerticalPitch
from matplotlib import animation
import matplotlib.pyplot as plt
from pathlib import Path
import imageio.v2 as iio
from tqdm import tqdm
import pandas as pd
import matplotlib
import zipfile
import shutil
import time
import json
import enum

# set pandas option to show columns of a dataframe
pd.set_option('display.max_columns', None)

In [4]:
import pandas as pd
import json
from IPython.display import display

### match_ids

Pick matches from the table

In [8]:
root = '../data/data_euro/'
match_ids = pd.read_csv(root + 'matchids.csv')

In [9]:
match_ids

,wyscout,skillcorner,date,home,result,away
0,5414103,NaN,2023-03-23,Kazakhstan,1-2,Slovenia
1,5414104,NaN,2023-03-23,Italy,1-2,England
2,5414105,NaN,2023-03-23,North Macedonia,2-1,Malta
3,5414106,NaN,2023-03-23,Denmark,3-1,Finland
4,5414107,NaN,2023-03-23,San Marino,0-2,Northern Ireland
...,...,...,...,...,...,...
236,5414329,1381512.0,2023-11-21,Wales,1-1,Turkey
237,5414330,1370341.0,2023-11-21,Romania,1-0,Switzerland
238,5414331,1381511.0,2023-11-21,Andorra,0-2,Israel
239,5414332,NaN,2023-11-21,Kosovo,0-1,Belarus


In [10]:
match_ids[(match_ids['home'] == 'France') | (match_ids['away'] == 'France')]

,wyscout,skillcorner,date,home,result,away
8,5414111,952209.0,2023-03-24,France,4-0,Netherlands
33,5414135,952208.0,2023-03-27,Republic of Ireland,0-1,France
52,5414152,NaN,2023-06-16,Gibraltar,0-3,France
77,5414176,952207.0,2023-06-19,France,1-0,Greece
98,5414196,NaN,2023-09-07,France,2-0,Republic of Ireland
144,5522737,NaN,2023-09-12,Germany,2-1,France
153,5414250,1381443.0,2023-10-13,Netherlands,1-2,France
213,5414307,1381494.0,2023-11-18,France,14-0,Gibraltar
233,5414326,1381515.0,2023-11-21,Greece,2-2,France


In [12]:
# france vs netherlands

match_id_wy = 5414111
match_id_sk = 952209


### Load wyscout [event data]

In [13]:
# wyscout
base_path = '../data/data_euro/primer_on_project_data/input/wyscout'
file_name = '{}.json'.format(match_id_wy)

In [14]:
def wyscout_json_to_df(file_name):
    with open(base_path+ '/' +file_name, encoding='utf8') as f:
        js = json.load(f)
        df = pd.json_normalize(js['events'])
    return df

df_events = wyscout_json_to_df(file_name)
#save df to csv
df_events.to_csv("df_events.csv")
df_events

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type.primary,type.secondary,location.x,location.y,team.id,team.name,team.formation,opponentTeam.id,opponentTeam.name,opponentTeam.formation,player.id,player.name,player.position,pass.accurate,pass.angle,pass.height,pass.length,pass.recipient.id,pass.recipient.name,pass.recipient.position,pass.endLocation.x,pass.endLocation.y,possession.id,possession.duration,possession.types,possession.eventsNumber,possession.eventIndex,possession.startLocation.x,possession.startLocation.y,possession.endLocation.x,possession.endLocation.y,possession.team.id,possession.team.name,possession.team.formation,possession.attack,pass,carry.progression,carry.endLocation.x,carry.endLocation.y,groundDuel.opponent.id,groundDuel.opponent.name,groundDuel.opponent.position,groundDuel.duelType,groundDuel.keptPossession,groundDuel.progressedWithBall,groundDuel.stoppedProgress,groundDuel.recoveredPossession,groundDuel.takeOn,groundDuel.side,groundDuel.relatedDuelId,possession.attack.withShot,possession.attack.withShotOnGoal,possession.attack.withGoal,possession.attack.flank,possession.attack.xg,shot.bodyPart,shot.isGoal,shot.onTarget,shot.goalZone,shot.xg,shot.postShotXg,shot.goalkeeperActionId,shot.goalkeeper.id,shot.goalkeeper.name,possession,aerialDuel.opponent.id,aerialDuel.opponent.name,aerialDuel.opponent.position,aerialDuel.opponent.height,aerialDuel.firstTouch,aerialDuel.height,aerialDuel.relatedDuelId,shot.goalkeeper,infraction.yellowCard,infraction.redCard,infraction.type,infraction.opponent.id,infraction.opponent.name,infraction.opponent.position,infraction.opponent
0,1587492614,5414111,1H,0,1,00:00:01.912,3.912638,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",51,50,664,Netherlands,4-2-3-1,4418,France,4-3-3,118,M. Depay,CF,True,114.0,None,24.0,625.0,M. de Roon,RDMF,42.0,83.0,1.587493e+09,110.275779,[],43.0,0.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1587492615,5414111,1H,0,5,00:00:05.782,7.782226,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",42,83,664,Netherlands,4-2-3-1,4418,France,4-3-3,625,M. de Roon,RDMF,True,-133.0,None,21.0,476361.0,L. Geertruida,RCB,29.0,60.0,1.587493e+09,110.275779,[],43.0,1.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1587492616,5414111,1H,0,8,00:00:08.951,10.951834,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",29,60,664,Netherlands,4-2-3-1,4418,France,4-3-3,476361,L. Geertruida,RCB,True,-100.0,None,22.0,370.0,V. van Dijk,LCB,25.0,28.0,1.587493e+09,110.275779,[],43.0,2.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1587492617,5414111,1H,0,13,00:00:13.151,15.151013,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",25,28,664,Netherlands,4-2-3-1,4418,France,4-3-3,370,V. van Dijk,LCB,True,90.0,None,16.0,476361.0,L. Geertruida,RCB,25.0,51.0,1.587493e+09,110.275779,[],43.0,3.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1587492618,5414111,1H,0,14,00:00:14.332,16.33218,1.587493e+09,NaN,NaN,NaN,NaN,NaN,touch,[carry],25,51,664,Netherlands,4-2-3-1,4418,France,4-3-3,476361,L. Geertruida,RCB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.587493e+09,110.275779,[]

In [15]:
# pass columns
pass_columns = [col for col in df_events.keys() if 'pass' in col]
pass_columns

['pass.accurate',
 'pass.angle',
 'pass.height',
 'pass.length',
 'pass.recipient.id',
 'pass.recipient.name',
 'pass.recipient.position',
 'pass.endLocation.x',
 'pass.endLocation.y',
 'pass']

In [16]:
df_events[pass_columns]

,pass.accurate,pass.angle,pass.height,pass.length,pass.recipient.id,pass.recipient.name,pass.recipient.position,pass.endLocation.x,pass.endLocation.y,pass
0,True,114.0,None,24.0,625.0,M. de Roon,RDMF,42.0,83.0,NaN
1,True,-133.0,None,21.0,476361.0,L. Geertruida,RCB,29.0,60.0,NaN
2,True,-100.0,None,22.0,370.0,V. van Dijk,LCB,25.0,28.0,NaN
3,True,90.0,None,16.0,476361.0,L. Geertruida,RCB,25.0,51.0,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# accurate passes
mask_acc_pass = df_events[pass_columns]['pass.accurate']==True
df_events[pass_columns][mask_acc_pass]

,pass.accurate,pass.angle,pass.height,pass.length,pass.recipient.id,pass.recipient.name,pass.recipient.position,pass.endLocation.x,pass.endLocation.y,pass
0,True,114.0,None,24.0,625.0,M. de Roon,RDMF,42.0,83.0,NaN
1,True,-133.0,None,21.0,476361.0,L. Geertruida,RCB,29.0,60.0,NaN
2,True,-100.0,None,22.0,370.0,V. van Dijk,LCB,25.0,28.0,NaN
3,True,90.0,None,16.0,476361.0,L. Geertruida,RCB,25.0,51.0,NaN
5,True,-95.0,None,23.0,370.0,V. van Dijk,LCB,30.0,17.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1802,True,3.0,None,22.0,56.0,D. Klaassen,AMF,61.0,91.0,NaN
1803,True,-82.0,None,15.0,221069.0,W. Weghorst,CF,63.0,69.0,NaN
1807,True,-74.0,None,29.0,134102.0,N. Aké,LB,72.0,30.0,NaN
1808,True,-45.0,None,12.0,118.0,M. Depay,LAMF,80.0,17.0,NaN


In [18]:
# team ids

df_events['team.id']

# map player ids to team ids
player_team_map = df_events[['player.id', 'team.id', 'team.name', 'player.name']].drop_duplicates()
player_team_map

,player.id,team.id,team.name,player.name
0,118,664,Netherlands,M. Depay
1,625,664,Netherlands,M. de Roon
2,476361,664,Netherlands,L. Geertruida
3,370,664,Netherlands,V. van Dijk
7,33,664,Netherlands,J. Cillessen
12,134102,664,Netherlands,N. Aké
14,491458,664,Netherlands,J. Timber
33,535816,664,Netherlands,K. Taylor
35,209117,4418,France,A. Rabiot
37,353833,4418,France,K. Mbappé


In [19]:
player_id0 = df_events[df_events['player.id']==0]
player_id0

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type.primary,type.secondary,location.x,location.y,team.id,team.name,team.formation,opponentTeam.id,opponentTeam.name,opponentTeam.formation,player.id,player.name,player.position,pass.accurate,pass.angle,pass.height,pass.length,pass.recipient.id,pass.recipient.name,pass.recipient.position,pass.endLocation.x,pass.endLocation.y,possession.id,possession.duration,possession.types,possession.eventsNumber,possession.eventIndex,possession.startLocation.x,possession.startLocation.y,possession.endLocation.x,possession.endLocation.y,possession.team.id,possession.team.name,possession.team.formation,possession.attack,pass,carry.progression,carry.endLocation.x,carry.endLocation.y,groundDuel.opponent.id,groundDuel.opponent.name,groundDuel.opponent.position,groundDuel.duelType,groundDuel.keptPossession,groundDuel.progressedWithBall,groundDuel.stoppedProgress,groundDuel.recoveredPossession,groundDuel.takeOn,groundDuel.side,groundDuel.relatedDuelId,possession.attack.withShot,possession.attack.withShotOnGoal,possession.attack.withGoal,possession.attack.flank,possession.attack.xg,shot.bodyPart,shot.isGoal,shot.onTarget,shot.goalZone,shot.xg,shot.postShotXg,shot.goalkeeperActionId,shot.goalkeeper.id,shot.goalkeeper.name,possession,aerialDuel.opponent.id,aerialDuel.opponent.name,aerialDuel.opponent.position,aerialDuel.opponent.height,aerialDuel.firstTouch,aerialDuel.height,aerialDuel.relatedDuelId,shot.goalkeeper,infraction.yellowCard,infraction.redCard,infraction.type,infraction.opponent.id,infraction.opponent.name,infraction.opponent.position,infraction.opponent
62,1587492672,5414111,1H,3,40,00:03:40.577,222.577357,NaN,NaN,NaN,NaN,NaN,NaN,game_interruption,[ball_out],100,91,664,Netherlands,4-2-3-1,4418,France,4-3-3,0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.587493e+09,31.7655745,[],14.0,13.0,50.0,50.0,100.0,91.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,1587496709,5414111,1H,4,7,00:04:07.472,249.472953,1.587493e+09,NaN,NaN,NaN,NaN,NaN,duel,[aerial_duel],7,41,4418,France,4-3-3,664,Netherlands,4-2-3-1,0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.587493e+09,2.2555255,"[corner, set_piece_attack, attack]",5.0,3.0,100.0,100.0,79.0,35.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,right,0.0077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,476361.0,L. Geertruida,RCB,185.0,True,NaN,1.587493e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,1587492676,5414111,1H,4,14,00:04:14.372,256.372679,NaN,NaN,NaN,NaN,NaN,NaN,game_interruption,[ball_out],100,63,664,Netherlands,4-2-3-1,4418,France,4-3-3,0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,1587492678,5414111,1H,4,33,00:04:33.230,275.230958,NaN,NaN,NaN,NaN,NaN,NaN,game_interruption,[ball_out],65,0,664,Netherlands,4-2-3-1,4418,France,4-3-3,0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.587494e+09,3.0699,"[set_piece_attack, free_kick]",3.0,2.0,0.0,49.0,35.0,100.0,4418.0,France,4-3-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,1587492774,5414111,1H,8,36,00:08:36.329,518.329036,NaN,NaN,NaN,NaN,NaN,NaN,game_interruption,[ball_out],100,88,664,Netherlands,4-2-3-1,4418,France,4-3-3,0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.587493e+09,23.095564,[attack],11.0,10.0,51.0,51.0,100.0,88.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,Fal

### Load Skillcorner

In [20]:
# Define the directories
input_dir = Path("../data/data_euro/primer_on_project_data/input/skillcorner")

# data_dir is where the processed raw data will be stored in
data_dir = Path("data")

# Create the data_dir if it does not exist
if not data_dir.exists():
    data_dir.mkdir()

# You can adjust the match_id according to your assigned task
match_id = match_id_sk

# skillcorner zip file path
zip_file = input_dir / f"{match_id}.zip"

# Extract the zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(input_dir)

# match metadata and tracking data file paths
metadata_file = input_dir / f"{match_id}.jsonl"
tracking_file = input_dir / f"{match_id}_tracking_extrapolated.jsonl"

# game intelligence data file paths
physical_file = input_dir / f"{match_id}_physical.json"
passes_file = input_dir / f"{match_id}_passes.json"
on_ball_pressures_file = input_dir / f"{match_id}_on_ball_pressures.json"
off_ball_runs_file = input_dir / f"{match_id}_off_ball_runs.json"

In [21]:
from skillcorner import *

In [22]:
skillcorner = SkillCorner()
skillcorner.load(match_id, metadata_file, tracking_file, physical_file, passes_file, on_ball_pressures_file, off_ball_runs_file, data_dir)

Metadata 952209 added ...
PlayDirection 952209 added ...
Tracking 952209 added ...
Visible Area 952209 added ...
Phase 952209 added ...
Lineup 952209 added ...
physical 952209 added ...
passes 952209 added ...
on_ball_pressures 952209 added ...
off_ball_runs 952209 added ...
Execution Time is 00:10


In [23]:
metadata_df = pd.read_csv(data_dir / f"{match_id}_metadata.csv")
play_direction_df = pd.read_csv(data_dir / f"{match_id}_play_direction.csv")
phase_df = pd.read_csv(data_dir / f"{match_id}_phase.csv")
lineup_df = pd.read_csv(data_dir / f"{match_id}_lineup.csv")
tracking_df = pd.read_csv(data_dir / f"{match_id}_tracking.csv")
visible_area_df = pd.read_csv(data_dir / f"{match_id}_visible_area.csv")
physical_df = pd.read_csv(data_dir / f"{match_id}_physical.csv")
passes_df = pd.read_csv(data_dir / f"{match_id}_passes.csv")
on_ball_pressures_df = pd.read_csv(data_dir / f"{match_id}_on_ball_pressures.csv")
off_ball_runs_df = pd.read_csv(data_dir / f"{match_id}_off_ball_runs.csv")
max_ball_z = int(tracking_df.z.max()) # what is the maximum ball height in meters that we see in our dataset.

In [24]:
metadata_df

,match_id,match_date,competition,season,home_team,away_team,home_score,away_score,home_team_jersey_color,away_team_jersey_color,home_team_number_color,away_team_number_color,home_team_coach,away_team_coach,pitch_name,pitch_length,pitch_width,provider,fps
0,952209,03/24/2023 19:45,European Championship Qualifiers,2023/2024,France,Netherland,4,0,#fafafa,#ff824c,#000000,#000000,Didier Deschamps,Frank De Boer,Stade de France,105.0,68.0,SkillCorner,10


In [25]:
lineup_df

,match_id,team_name,player_id,player_first_name,player_last_name,player_shirt_number,player_position,player_birthdate,start_time,end_time,yellow_card,red_card,injured,goal,own_goal
0,952209,Netherland,14397,Tyrell,Malacia,16,Right Wing Back,1999-08-17,01:26:59,NaN,0,0,False,0,0
1,952209,Netherland,12654,Donyell,Malen,7,Right Winger,1999-01-19,01:06:22,NaN,0,0,False,0,0
2,952209,Netherland,2548,Davy,Klaassen,14,Attacking Midfield,1993-02-21,01:06:22,NaN,0,0,False,0,0
3,952209,Netherland,2106,Daley,Blind,17,Left Midfield,1990-03-09,01:06:22,NaN,1,0,False,0,0
4,952209,Netherland,10916,Wout,Weghorst,9,Center Forward,1992-08-07,00:32:38,NaN,0,0,False,0,0
5,952209,France,20191,Khéphren,Thuram-Ulien,18,Left Midfield,2001-03-26,01:28:40,NaN,0,0,False,0,0
6,952209,France,12229,Youssouf,Fofana,6,Right Midfield,1999-01-10,01:16:24,NaN,0,0,False,0,0
7,952209,France,8206,Olivier,Giroud,9,Center Forward,1986-09-30,01:16:24,NaN,0,0,False,0,0
8,952209,France,22103,Eduardo,Camavinga,21,Defensive Midfield,2002-11-10,01:16:24,NaN,0,0,False,0,0
9,952209,France,12246,Moussa,Diaby,19,Right Forward,1999-07-07,01:06:46,NaN,0,0,False,0,0


# Sync: wyscout - skillcorner

## Team-id

In [40]:
# map team teamnames to team ids btw wyscout and skillcorner
team_name_sk = lineup_df[['team_name']].drop_duplicates()
team_name_sk.columns = ['team_name_sk']
print(team_name_sk)

df_map_wy = df_events[['team.name','team.id']].drop_duplicates()
# rename columns to team_name_wyscout and team_id_wyscout
df_map_wy.columns = ['team_name_wyscout', 'team_id_wyscout']
print(df_map_wy)




  team_name_sk
0   Netherland
5       France
   team_name_wyscout  team_id_wyscout
0        Netherlands              664
35            France             4418


In [31]:
lineup_df.iloc[0]

match_id                        952209
team_name                   Netherland
player_id                        14397
player_first_name               Tyrell
player_last_name               Malacia
player_shirt_number                 16
player_position        Right Wing Back
player_birthdate            1999-08-17
start_time                    01:26:59
end_time                           NaN
yellow_card                          0
red_card                             0
injured                          False
goal                                 0
own_goal                             0
Name: 0, dtype: object

In [66]:
lineup_df.keys()

Index(['match_id', 'team_name', 'player_id', 'player_first_name',
       'player_last_name', 'player_shirt_number', 'player_position',
       'player_birthdate', 'start_time', 'end_time', 'yellow_card', 'red_card',
       'injured', 'goal', 'own_goal'],
      dtype='object')

## Player-id

In [68]:
# get player id from skillcorner
# player_id_sk = lineup_df[lineup_df['team_name']=='Netherland'][['player_id', 'player_first_name', 'player_last_name']]
player_id_sk = lineup_df[['player_id', 'player_first_name', 'player_last_name', 'team_name']]
player_id_sk.columns = ['player_id_sk', 'player_first_name_sk', 'player_last_name_sk', 'team_name_sk']

print(len(player_id_sk))
player_id_sk

32


,player_id_sk,player_first_name_sk,player_last_name_sk,team_name_sk
0,14397,Tyrell,Malacia,Netherland
1,12654,Donyell,Malen,Netherland
2,2548,Davy,Klaassen,Netherland
3,2106,Daley,Blind,Netherland
4,10916,Wout,Weghorst,Netherland
5,20191,Khéphren,Thuram-Ulien,France
6,12229,Youssouf,Fofana,France
7,8206,Olivier,Giroud,France
8,22103,Eduardo,Camavinga,France
9,12246,Moussa,Diaby,France


In [69]:
# get player id and name from wyscout
# player_id_wy = player_team_map[player_team_map['team.name']=='Netherlands'][['player.id', 'player.name']]
player_id_wy = player_team_map[['player.id', 'player.name', 'team.id', 'team.name']]
# filter out player_id=0
player_id_wy = player_id_wy[player_id_wy['player.id']!=0]

player_id_wy.columns = ['player_id_wy', 'player_name_wy', 'team_id_wy', 'team_name_wy']

print(len(player_id_wy))
player_id_wy

32


,player_id_wy,player_name_wy,team_id_wy,team_name_wy
0,118,M. Depay,664,Netherlands
1,625,M. de Roon,664,Netherlands
2,476361,L. Geertruida,664,Netherlands
3,370,V. van Dijk,664,Netherlands
7,33,J. Cillessen,664,Netherlands
12,134102,N. Aké,664,Netherlands
14,491458,J. Timber,664,Netherlands
33,535816,K. Taylor,664,Netherlands
35,209117,A. Rabiot,4418,France
37,353833,K. Mbappé,4418,France


In [78]:
# len wyscout should match len skillcorner
assert len(player_id_wy) == len(player_id_sk)

# match player id in wyscout and skillcorner
player_id_wy_sorted = player_id_wy.sort_values(by='player_name_wy')

player_id_sk_sorted = player_id_sk.sort_values(by=['player_first_name_sk', 'player_last_name_sk'])




In [79]:
player_id_wy_sorted

,player_id_wy,player_name_wy,team_id_wy,team_name_wy
43,3682,A. Griezmann,4418,France
35,209117,A. Rabiot,4418,France
100,480145,A. Tchouaméni,4418,France
1355,38,D. Blind,664,Netherlands
1356,56,D. Klaassen,664,Netherlands
1377,365446,D. Malen,664,Netherlands
61,352988,D. Upamecano,4418,France
1523,599551,E. Camavinga,4418,France
137,116,G. Wijnaldum,664,Netherlands
65,393247,I. Konaté,4418,France


In [77]:
player_id_sk_sorted

,player_id_sk,player_first_name_sk,player_last_name_sk,team_name_sk
27,277,Adrien,Rabiot,France
30,946,Antoine,Griezmann,France
29,12218,Aurélien,Tchouaméni,France
3,2106,Daley,Blind,Netherland
2,2548,Davy,Klaassen,Netherland
26,11516,Dayot,Upamecano,France
1,12654,Donyell,Malen,Netherland
8,22103,Eduardo,Camavinga,France
15,3837,Georginio,Wijnaldum,Netherland
25,16320,Ibrahima,Konaté,France


In [111]:
# match sorted skillcorner row with wyscout row
# player_id_map = pd.concat([player_id_sk_sorted.reset_index(drop=True), player_id_wy_sorted.reset_index(drop=True)], axis=1)
# player_id_maph

In [95]:
from fuzzywuzzy import fuzz
from Levenshtein import distance as levenshtein_distance
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')



/Users/alpayozkan/opt/anaconda3/envs/soccer/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [110]:
# sim_arr_lev = []
# sim_arr_fuzz = []

sim_arr = []
match_arr = []
# for each row
for row_sk in player_id_sk_sorted.iterrows():
    sim_arr_cosine = []

    surname = row_sk[1]['player_last_name_sk']
    surname = surname.lower()
    
    name = row_sk[1]['player_first_name_sk']
    name = name.lower()

    full_name = name + ' ' + surname

    # find the corresponding row in wyscout
    for row_wy in player_id_wy_sorted.iterrows():
        
        name_wy = row_wy[1]['player_name_wy']
        name_wy = name_wy.lower()

        # Using Levenshtein Distance
        lev_distance = levenshtein_distance(full_name, name_wy)
        # Using Fuzzy Matching
        similarity_ratio = fuzz.ratio(full_name, name_wy)

        # Encode sentences
        embeddings1 = model.encode(full_name, convert_to_tensor=True)
        embeddings2 = model.encode(name_wy, convert_to_tensor=True)
        # Compute cosine similarity
        cosine_similarity = util.pytorch_cos_sim(embeddings1, embeddings2)

        # print('name_wy | full_name: {} | {}'.format(name_wy, full_name))
        # print(f"Levenshtein Distance: {lev_distance}")
        # print(f"Fuzzy Matching Similarity Ratio: {similarity_ratio}%")
        # print(f"Cosine Similarity: {cosine_similarity.item()}")

        # sim_arr_lev.append(lev_distance)
        # sim_arr_fuzz.append(similarity_ratio)
        sim_arr_cosine.append(cosine_similarity.item())

    sim_arr_cosine = np.array(sim_arr_cosine)
    sim_arr.append(sim_arr_cosine)
    match_arr.append(np.argmax(sim_arr_cosine))
    
# sim_arr_lev = np.array(sim_arr_lev)
# sim_arr_fuzz = np.array(sim_arr_fuzz)
# sim_arr_cosine = np.array(sim_arr_cosine)

# res_lev = np.argmin(sim_arr_lev)
# res_fuzz = np.argmax(sim_arr_fuzz)
# res_cosine = np.argmax(sim_arr_cosine)

# print(res_lev)
# print(res_fuzz)
# print(res_cosine)

# print(full_name)
# print(player_id_wy_sorted.iloc[res_cosine])


In [120]:
# check duplicates in the array
match_arr = np.array(match_arr)
unique, counts = np.unique(match_arr, return_counts=True)

# assert no duplicates => all players are matched
assert len(unique) == len(match_arr)

# match skillcorner and wyscout rows

player_id_map = pd.concat([player_id_sk_sorted.reset_index(drop=True), player_id_wy_sorted.iloc[match_arr].reset_index(drop=True)], axis=1)
player_id_map



,player_id_sk,player_first_name_sk,player_last_name_sk,team_name_sk,player_id_wy,player_name_wy,team_id_wy,team_name_wy
0,277,Adrien,Rabiot,France,209117,A. Rabiot,4418,France
1,946,Antoine,Griezmann,France,3682,A. Griezmann,4418,France
2,12218,Aurélien,Tchouaméni,France,480145,A. Tchouaméni,4418,France
3,2106,Daley,Blind,Netherland,38,D. Blind,664,Netherlands
4,2548,Davy,Klaassen,Netherland,56,D. Klaassen,664,Netherlands
5,11516,Dayot,Upamecano,France,352988,D. Upamecano,4418,France
6,12654,Donyell,Malen,Netherland,365446,D. Malen,664,Netherlands
7,22103,Eduardo,Camavinga,France,599551,E. Camavinga,4418,France
8,3837,Georginio,Wijnaldum,Netherland,116,G. Wijnaldum,664,Netherlands
9,16320,Ibrahima,Konaté,France,393247,I. Konaté,4418,France
